# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
conda install -c conda-forge gmaps

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
path = "../WeatherPy/cities.csv"
data_df = pd.read_csv(path)
data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kavieng,-2.5744,150.7967,82.11,75,100,8.12,PG,1648022672
1,Ushuaia,-54.8000,-68.3000,37.06,98,90,5.08,AR,1648022585
2,Jalu,29.0331,21.5482,53.78,42,0,12.50,LY,1648022673
3,Ribeira Grande,38.5167,-28.7000,61.20,63,75,6.91,PT,1648022674
4,Teya,60.3778,92.6267,26.29,71,100,7.61,RU,1648022675
...,...,...,...,...,...,...,...,...,...
567,Kasongo-Lunda,-6.4667,16.8167,74.01,74,100,3.33,CD,1648023099
568,Brae,60.3964,-1.3530,47.23,90,92,11.45,GB,1648023099
569,Karatau,43.1667,70.4667,34.14,100,100,2.15,KZ,1648023100
570,Mékhé,15.1097,-16.6218,65.23,77,3,15.12,SN,1648023101


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure gmaps.
gmaps.configure(api_key=g_key)

In [8]:
# Store latitude and longitude in locations
locations = data_df[["Lat", "Lng"]]
# Fill NaN values and convert to float : weight
humidity = data_df["Humidity"].astype(float)
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heatmap_layer)
plt.savefig("Heatmap.png")
fig 

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
cloud_data = data_df[data_df["Cloudiness"] == 0]
# Filter temperature lower than 80 degrees but higher than 70:
temp_data = cloud_data[(cloud_data["Max Temp"] > 70) & (cloud_data["Max Temp"] < 80)]

# Filter Wind speed
wind_data = temp_data[temp_data["Wind Speed"] < 10]

# Drop any rows will null values:
wind_data = wind_data.dropna()
wind_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,La Reforma,18.1000,-91.0167,71.85,84,0,7.90,MX,1648022684
40,Dārāb,28.7519,54.5444,78.46,12,0,3.83,IR,1648022701
54,Busselton,-33.6500,115.3333,77.45,37,0,4.09,AU,1648022647
177,Kruisfontein,-34.0033,24.7314,75.88,47,0,2.37,ZA,1648022801
381,Plettenberg Bay,-34.0527,23.3716,74.35,64,0,4.12,ZA,1648022381
571,Fortuna Foothills,32.6578,-114.4119,71.08,23,0,5.75,US,1648023102


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = wind_data
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
18,La Reforma,18.1000,-91.0167,71.85,84,0,7.90,MX,1648022684,
40,Dārāb,28.7519,54.5444,78.46,12,0,3.83,IR,1648022701,
54,Busselton,-33.6500,115.3333,77.45,37,0,4.09,AU,1648022647,
177,Kruisfontein,-34.0033,24.7314,75.88,47,0,2.37,ZA,1648022801,
381,Plettenberg Bay,-34.0527,23.3716,74.35,64,0,4.12,ZA,1648022381,
571,Fortuna Foothills,32.6578,-114.4119,71.08,23,0,5.75,US,1648023102,


In [29]:
# Set params:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Loop through to identify hotels:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params['location'] = f'{lat},{lng}'
    
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.


In [30]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
18,La Reforma,18.1000,-91.0167,71.85,84,0,7.90,MX,1648022684,NA
40,Dārāb,28.7519,54.5444,78.46,12,0,3.83,IR,1648022701,Naghshe shapoor Hotel
54,Busselton,-33.6500,115.3333,77.45,37,0,4.09,AU,1648022647,The Sebel Busselton
177,Kruisfontein,-34.0033,24.7314,75.88,47,0,2.37,ZA,1648022801,NA
381,Plettenberg Bay,-34.0527,23.3716,74.35,64,0,4.12,ZA,1648022381,The Plettenberg Hotel
571,Fortuna Foothills,32.6578,-114.4119,71.08,23,0,5.75,US,1648023102,Comfort Inn & Suites Yuma I-8


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# Add marker layer ontop of heat map

# Display figure
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))